In [1]:
import numpy as np
import pandas as pd

from statsmodels.api import tsa
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing

In [2]:
def _fit_arima_for_row(time_series, **kwargs):
    try:
        order = tsa.arma_order_select_ic(time_series, ic=["aic"], trend="nc").aic_min_order
        model = ARIMA(time_series, order=(order[0], 0, order[1]))
        res = model.fit(disp=-1, method="css", **kwargs)
    except np.linalg.LinAlgError:
        # fallback to HW model if convergence error happens
        return _fit_holt_winters_for_row(time_series, **kwargs)
    except ValueError:
        return _fit_holt_winters_for_row(time_series, **kwargs)

    return res, model

In [3]:
def _fit_holt_winters_for_row(time_series, **kwargs):
    model = ExponentialSmoothing(time_series)
    res = model.fit(**kwargs)

    return res, model

In [4]:
time_series = np.array([1, 2, 1, 2, 1, 2, 1, 2, 1, 2]).astype(float)

In [ ]:
res, model = _fit_holt_winters_for_row(time_series)
prediction = res.predict(start=len(time_series), end=len(time_series))
prediction